This notebook will replicate H2 (minimal basis, B-K) energy evaluation using pytket, pytket_qiskit and pytket_honeywell

python~=3.7
requirements:
pytket==0.4.3
pytket_qiskit>0.3.4
pytket_honeywell==0.0.1

sympy
numpy

In [1]:
import os
from functools import reduce
import operator
import copy
from collections import Counter

import sympy
import numpy as np

from pytket.circuit import Circuit
from pytket.extensions.qiskit import AerBackend, IBMQBackend
from pytket.extensions.honeywell import HoneywellBackend
from pytket.circuit import PauliExpBox, fresh_symbol
from pytket.pauli import Pauli
from pytket.utils.measurements import append_pauli_measurement
from pytket.utils import expectation_from_counts
from pytket.passes import DecomposeBoxes, SynthesiseIBM


from h2_hamiltonians import bond_length_hams


Generate ansatz circuit using pytket using known operators

In [2]:
ansatz = Circuit(4,4)
param = fresh_symbol("t")
coeff = -2*param/sympy.pi
box = PauliExpBox((Pauli.Y, Pauli.Z, Pauli.X, Pauli.Z), coeff)
ansatz.X(0)
ansatz.add_pauliexpbox(box, ansatz.qubits)
DecomposeBoxes().apply(ansatz)


True

Set up hamiltonian processing code

In [3]:
contracted_measurement_bases = ('Z', 'X', 'Y')

def get_energy_from_counts(coeff_shots): 
    return sum(coeff * expectation_from_counts(shots) for coeff, shots in coeff_shots)

def operator_only_counts(counts, operator_qubs):
    filterstate = lambda s: tuple(np.array(s)[list(operator_qubs)])
    filtereredcounters = (Counter({filterstate(state):count}) for state, count in counts.items())
    return reduce(operator.add, filtereredcounters)

def submit_hamiltonian(state_circ, measurementops, n_shots, backend, base_name='ansatz'):

    circuits = []
    circuit_names = []
    for entry, basis in zip(measurementops,contracted_measurement_bases):
        meas_circ = state_circ.copy()
        append_pauli_measurement(entry, meas_circ)
        backend.compile_circuit(meas_circ)
        circuit_names += [f'{base_name}_{basis}']
        circuits.append(meas_circ)

    print(circuit_names)
    return backend.process_circuits(circuits, n_shots=n_shots)


def calculate_hamiltonian(measured_counts, hamiltonian):
    hamcopy = copy.copy(hamiltonian)
    constant_coeff = hamcopy.pop(tuple())
    hamoptups, coeff_list = zip(*hamcopy.items())
    converted_shot_list = []
    
    measurement_mapper = lambda opdict: opdict[0] if 0 in opdict else 'Z'
    
    hamop2measop = {tp: measurement_mapper(dict(tp)) for tp in hamoptups}

    meascounts_map = dict(zip(contracted_measurement_bases, measured_counts))

    for (optup, coeff) in zip(hamoptups, coeff_list):
        op_qubs, _ = zip(*optup)
        selected_meas_counts = meascounts_map[hamop2measop[optup]]
        converted_shot_list.append(operator_only_counts(selected_meas_counts, op_qubs))

    return get_energy_from_counts (zip(coeff_list, converted_shot_list)) + constant_coeff

Set up minimal measurement bases required to measure whole hamiltonian

In [4]:
measurement_ops = [((0, 'Z'), (1, 'Z'), (2, 'Z'), (3, 'Z')),
((0, 'X'), (1, 'Z'), (2, 'X'), (3, 'Z')), ((0, 'Y'), (1, 'Z'), (2, 'Y'), (3, 'Z'))]

Set up honeywell API key and machine name

In [5]:
hwell_apikey = '<apikey>'
hwell_machine = "HQS-LT-1.0-APIVAL"

In [7]:
bond_length = 0.735
popt = bond_length_hams[bond_length]["optimal_parameter"]
hamopt = bond_length_hams[bond_length]["hamiltonian"] 

aerbackend = AerBackend()
backend = HoneywellBackend(hwell_apikey, device_name=hwell_machine, label='h2_exp')
# backend = aerbackend
backend = IBMQBackend('ibmq_burlington', hub='ibmq')
state_circuit = ansatz.copy()
state_circuit.symbol_substitution({param:popt})

sub_jobs = submit_hamiltonian(state_circuit, measurement_ops, 100, backend)
print(sub_jobs)
results = [backend.get_counts(job) for job in sub_jobs]

calculate_hamiltonian(results, hamopt)

['ansatz_Z', 'ansatz_X', 'ansatz_Y']
[('5e554bacd8204b0018fd539f', 0), ('5e554bacd8204b0018fd539f', 1), ('5e554bacd8204b0018fd539f', 2)]
Job Status: job has successfully run
Job Status: job has successfully run
Job Status: job has successfully run


-0.6357045412185943

In [ ]:

retrieve_ids = ['', '', '']
retrieved_results = [backend.get_counts(JobHandle(id)) for id in retrieve_ids]

calculate_hamiltonian(retrieved_results, hamopt)